Install neccessary packages 

In [2]:
from urllib import request as rq
import pandas as pd

Download the Dublin Bus data from https://data.gov.ie/organization/dublin-city-council?tags=Transport+and+Infrastructure

In [3]:
url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )

In [38]:
import zipfile as zip

with zip.ZipFile(busFile[0],  mode='r') as arc: 
    arc.extractall('./Data/Bus/Gz/')

Use a generator to load teh data into pandas data frame https://pandasninja.com/2019/04/how-to-read-lots-of-csv-files-easily-into-pandas/#:~:text=How%20to%20read%20lots%20of%20csv%20files%20easily,need%20...%204%204.%20Leverage%20regular%20expressions%20


In [3]:
import os 
files = os.listdir('./Data/Bus/Gz/')
DBfiles = [f for f in files if f.endswith('.gz')]
def load_Files(files):
    for f in files:
        yield pd.read_csv('./Data/Bus/Gz/' + f, compression='gzip')

df = pd.concat(load_Files(DBfiles))

KeyboardInterrupt: 

Download the weather station details https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv

{'.', '0', '1', '2', '3', 'c', 'g', 'i', 'r', 's', 'v', 'z'}

In [3]:
url = 'https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv'


Load the weather statio details for Dublin

In [6]:
weatherStations = pd.read_csv(url)
weatherStations.head()

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,(null)
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,(null)
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944


,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,(null)
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,(null)
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944
